In [ ]:
import os
from copy import deepcopy
from importlib import reload
from collections import namedtuple
import traceback

import numpy as np
import random
import argparse
from tqdm.auto import tqdm

import models
reload(models)
import models
import utils
reload(utils)
import utils

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.optim import Adam
from torchvision import datasets
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint, BatchSizeFinder

from pytorch_lightning.loggers import TensorBoardLogger
#cmd command: tensorboard --logdir=lightning_logs/

random.seed(123)

In [ ]:
DATA_DIR = 'data/tiny-imagenet-200'
TRAIN_DIR = os.path.join(DATA_DIR, 'train') 
VALID_DIR = os.path.join(DATA_DIR, 'val')
VALID_IMG_DIR = os.path.join(VALID_DIR, 'images')

In [ ]:
preprocess_transform_pretrain = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])
])

In [ ]:
categories = 50

In [ ]:
train_loader = utils.generate_dataloader(TRAIN_DIR, "train", categories=categories, 
                                  transform=preprocess_transform_pretrain, batch_size=256, unseen=False)
val_seen_loader = utils.generate_dataloader(VALID_IMG_DIR, "val_seen", categories=categories, 
                                  transform=preprocess_transform_pretrain, batch_size=256, unseen=False)
val_unseen_loader = utils.generate_dataloader(VALID_IMG_DIR, "val_unseen", categories=categories, 
                                  transform=preprocess_transform_pretrain, batch_size=256, unseen=True)

In [ ]:
train_embedding_loader = utils.generate_embedding_loader(TRAIN_DIR, "train", categories=categories, 
                                  transform=preprocess_transform_pretrain, batch_size=16, unseen=False)
val_seen_embedding_loader = utils.generate_embedding_loader(VALID_IMG_DIR, "val_seen", categories=categories, 
                                  transform=preprocess_transform_pretrain, batch_size=16, unseen=False)
val_unseen_embedding_loader = utils.generate_embedding_loader(VALID_IMG_DIR, "val_unseen", categories=categories, 
                                  transform=preprocess_transform_pretrain, batch_size=16, unseen=True)

In [ ]:
train_triplet_loader = utils.generate_triplet_loader(TRAIN_DIR, "train", categories=categories, 
                                  transform=preprocess_transform_pretrain, batch_size=64, unseen=False)
val_triplet_loader = utils.generate_triplet_loader(VALID_IMG_DIR, "val_seen", categories=categories, 
                                  transform=preprocess_transform_pretrain, batch_size=64, unseen=False)

## Pytorch Lightning

### Siamese Sigmoid Model

In [ ]:
try:
    for lr in [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]:
        callbacks = [
            ModelCheckpoint(monitor='val_loss', save_top_k=1, dirpath=None),
            EarlyStopping(monitor="val_loss", min_delta=0.0, patience=10, verbose=False, mode="min")
            ]
        logger = TensorBoardLogger("lightning_logs", name=f"Sigmoid-50-batchsize-256-lr-{str(lr)}")
        trainer = pl.Trainer(max_epochs=50, accelerator="gpu", auto_lr_find=False, gradient_clip_val=0.0, log_every_n_steps=50, logger=logger, callbacks=callbacks)
        model = models.SiameseSigmoid(lr=lr, pretrained=True)
        combined_val_loader = model.val_dataloader(val_seen_loader, val_unseen_loader)
        trainer.fit(model, train_loader, combined_val_loader)
except:
    traceback.print_exc()

### Siamese Cosine Model

In [ ]:
try:
    for lr in [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]:
        callbacks = [
            ModelCheckpoint(monitor='val_loss', save_top_k=1, dirpath=None),
            EarlyStopping(monitor="val_loss", min_delta=0.0, patience=10, verbose=False, mode="min")
            ]
        logger = TensorBoardLogger("lightning_logs", name=f"Cosine-50-batchsize-256-no-pretrain-lr-{str(lr)}")
        trainer = pl.Trainer(max_epochs=50, accelerator="gpu", auto_lr_find=False, gradient_clip_val=0.0, log_every_n_steps=50, logger=logger, callbacks=callbacks)
        model = models.SiameseCosine(lr=lr, categories=categories, pretrained=True)
        combined_val_loader = model.val_dataloader(val_seen_loader, val_unseen_loader)
        trainer.fit(model, train_loader, combined_val_loader)
except:
    traceback.print_exc()